In [ ]:
from sqlalchemy import create_engine, desc
from idetect.model import db_url, Base, Article, Session, Status

# connect to the DB specified in the docker.env file
engine = create_engine(db_url())
Session.configure(bind=engine)

# create the DB schema, if it doesn't already exist
Base.metadata.create_all(engine)

In [ ]:
session = Session()

In [ ]:
from os.path import isfile

path = '/home/idetect/data/input_urls.csv'
url = 'https://www.dropbox.com/s/c2vzdzrljlrn3y0/input_urls.csv?dl=1'

if not isfile(path):
    with open(path, 'wb') as f:
        r = requests.get(url, stream=True)
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
import csv
from datetime import datetime

start = datetime.now()
i = 0
with open('/home/idetect/data/input_urls.csv') as f:
    c = csv.reader(f)
    for l in c:
        url_id, date, url = l
        if not url.startswith('http'):
            continue
        try:
            article1 = Article(url=url, url_id=url_id, status=Status.NEW)
            session.add(article1)
            session.commit()
        except:
            print("error on row {}: {}".format(i, l))
            raise
        i += 1
        if i % 1000 == 0:
            print("{} {}".format(i, url))
done_inserting = datetime.now()

In [ ]:
delta = done_inserting - start

In [ ]:
delta.seconds / 60 / 60